# Extracting the audio book data

In [136]:
import numpy as np
from sklearn import preprocessing
import tensorflow as tf
np.random.seed(9)
raw_csv_data=np.loadtxt('Audiobooks_data.csv',delimiter=',')
unscalled_inputs_all=raw_csv_data[:,1:-1] #except the 1st and last cloumn
targets_all=raw_csv_data[:,-1] #last column

# Balance the dataset

In [137]:
num_one_targets=list(targets_all).count(1) #number of ones in target
zero_target_counter=0
indices_to_remove=[]
#keeping as many as 0s as 1s (we will delete others)
#if number of zeros == number of ones then remove all further positions which contains zero value in targets
for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_target_counter += 1
        if zero_target_counter>num_one_targets:
            indices_to_remove.append(i)

unscalled_inputs_equal_priors=np.delete(unscalled_inputs_all,indices_to_remove,axis=0) #axis 0 of the indices_to_remove vector
targets_equal_priors=np.delete(targets_all,indices_to_remove,axis=0)

# Standardizing the inputs

In [138]:
scaled_inputs=preprocessing.scale(unscalled_inputs_equal_priors)

# Shuffel the data

In [139]:
indices_of_scaled_inputs=list(np.arange(scaled_inputs.shape[0]))
np.random.shuffle(indices_of_scaled_inputs) #shuffleing the indices

shuffled_inputs=scaled_inputs[indices_of_scaled_inputs]
shuffled_targets=targets_equal_priors[indices_of_scaled_inputs]


# Splitting the dataset into training,validation &testing

In [140]:
samples_count=shuffled_inputs.shape[0]

train_samples_count=int(0.8*samples_count)
validation_samples_count=int(0.1*samples_count)
test_samples_count=samples_count - train_samples_count - validation_samples_count

train_inputs=shuffled_inputs[:train_samples_count] # 1-80
train_targets=shuffled_targets[:train_samples_count]

validation_inputs=shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count] # 81-90
validation_targets=shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs=shuffled_inputs[train_samples_count+validation_samples_count:]#91-100
test_targets=shuffled_targets[train_samples_count+validation_samples_count:]


# Saving the 3 datasets in *npz format

In [141]:
np.savez('Audiobooks_data_train',inputs=train_inputs,targets=train_targets)
np.savez('Audiobooks_data_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Audiobooks_data_test',inputs=test_inputs,targets=test_targets)

# Create methods that handles batches

In [142]:
import numpy as np

class Audiobooks_Data_Reader():
    #dataset mandetory batch_size optional #dataset inputs+targets #dataset=train or validation or test 
    def __init__(self,dataset,batch_size=None): ### taking train or validation or test as input ###
        
        #dataset=train or validation or test
        npz=np.load('Audiobooks_data_{0}.npz'.format(dataset))
        
        self.inputs,self.targets=npz['inputs'].astype(np.float),npz['targets'].astype(np.int)
        
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
        ### creating number of batches wrt input

    # A method which loads the next batch
    def __next__(self): ###iterating each batch at a time ####
        
        if self.curr_batch >= self.batch_count:### iteration termination logic ###
                self.curr_batch = 0
                raise StopIteration()
       
        # You slice the dataset in batches and then the "next" function loads them one after the other
        #get each slice in each iteration
        ### creating each batch slice ###
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size) #each slice size
        batch_inputs = self.inputs[batch_slice]
        batch_targets = self.targets[batch_slice]
        
        self.curr_batch += 1
        
        ### hot encoding ###
        classes_num = 2
        ### list of all zeroes of length batch_targets.shape[0] ###
        targets_one_hot = np.zeros((batch_targets.shape[0], classes_num)) 
        ## we can use 1st 3 lines of next batch ##
        targets_one_hot[range(batch_targets.shape[0]), batch_targets] = 1 #batch_targets 0 or 1 #******

        return batch_inputs, targets_one_hot ### returning batched inputs and one hot encoded batched output ###
        
    def __iter__(self):
        return self

In [ ]:
#******

In [143]:
#### We can also use these 3 lines
#******
### means
'''
            (batch_targets.shape[0]=batched_target_length)0    1    
                                    .................................
                                            0             0    0 
                                            1             0    0 
                                            2             0    0 
                                            .             .
                                            .             .
                                            n             0    0 
                                            
            (batch_targets.shape[0]=batched_target_length)0    1    i  batch_targets[i]
                                    ................................................
                                            0             0    1 <=[0,      1]              =1
                                            1             1    0 <=[1,      0]              =1
                                            2             1    0 <=[2,      0]              =1
                                            .             .
                                            .             .
                                            n             0    1 <=>[n,     1]              =1
                
                for i in range(batch_targets.shape[0]):
                    all_zeroes[i,batch_targets]=1
                    
                means when batch_targets=1 at i=k instance =>all_zeroes[k,1]=1 || 0 encoded to >1[k 0] 0[k 1]
                and when batch_targets=0 at i=l instance   =>all_zeroes[l,0]=0 || 1 encoded to >0[l 0] 1[l 1]
        
       
'''

'\n            (batch_targets.shape[0]=batched_target_length)0    1    \n                                    .................................\n                                            0             0    0 \n                                            1             0    0 \n                                            2             0    0 \n                                            .             .\n                                            .             .\n                                            n             0    0 \n                                            \n            (batch_targets.shape[0]=batched_target_length)0    1    i  batch_targets[i]\n                                    ................................................\n                                            0             0    1 <=[0,      1]              =1\n                                            1             1    0 <=[1,      0]              =1\n                                            2        

# Main code

In [147]:
input_size=10
output_size=2
hidden_layer_size=100

#removes all previous stored graphs or data
tf.reset_default_graph()

inputs=tf.placeholder(tf.float32,[None,input_size])
targets=tf.placeholder(tf.int32,[None,output_size])

#for layer 1
weights_1=tf.get_variable('weights_1',[input_size,hidden_layer_size])
biases_1=tf.get_variable('biases_1',[hidden_layer_size])

outputs_1=tf.nn.relu(tf.matmul(inputs,weights_1)+biases_1) #layer one activation function relu

#for layer 2
weights_2=tf.get_variable('weights_2',[hidden_layer_size,hidden_layer_size])
biases_2=tf.get_variable('biases_2',[hidden_layer_size])

outputs_2=tf.nn.relu(tf.matmul(outputs_1,weights_2)+biases_2) #layer two activation function relu

#final layer 
weights_3=tf.get_variable('weights_3',[hidden_layer_size,output_size])
biases_3=tf.get_variable('biases_3',[output_size])

outputs=tf.matmul(outputs_2,weights_3)+biases_3 #output without activation function


#1st logit function then softmax then cross_entropy loss calculation
#numaricaly stable function when we deals with very small nnumbers
cross_entropy_loss=tf.nn.softmax_cross_entropy_with_logits(logits=outputs,labels=targets) #returns list of losses
mean_loss=tf.reduce_mean(cross_entropy_loss) #<<<<<<<<<<

#optimization method  Adam
optimizer=tf.train.AdamOptimizer(learning_rate=0.0001).minimize(mean_loss)#back_prob also performed here #<<<<<<<<<

#prediction accuracy
#equal returns 1 or 0 when values matched or not matched respectively
#output_equals_target is a vactor of 0's (missmatch) and 1's (match) 
#argmax returs the index(r,c) of the maximum value for each target and output list, we need only the column number 
output_equals_target=tf.equal(tf.arg_max(outputs,1),tf.arg_max(targets,1))#argmax retruns row &col num here 1 means only cloumn
accuracy=tf.reduce_mean(tf.cast(output_equals_target,tf.float32)) #<<<<< #cast->[0,1,1]=>[0.0,1.0,1.0]

#creating sessions
sess=tf.InteractiveSession()

#initialize all variables 
initializer=tf.global_variables_initializer() #<<<<<<<
sess.run(initializer)

#batching total 100 batches required
batch_size=100 #batches of max 100 entries


max_epochs=500

previous_validation_loss=9999999. #. makes an int to float 

train_data=Audiobooks_Data_Reader('train',batch_size)
validation_data=Audiobooks_Data_Reader('validation')


for epoch_counter in range(max_epochs):
    current_epoch_loss=0. #trainig loss 
    total_batch_loss=0.
    
    for batch_inputs,batch_targets in train_data:
        
        #getting minimized batch loss with backpropagation
        _,current_batch_loss=sess.run([optimizer,mean_loss],
                                      feed_dict={inputs:batch_inputs,targets:batch_targets}) 
        
        total_batch_loss += current_batch_loss
        
    current_epoch_loss = total_batch_loss/train_data.batch_count
    
    #validation after each epoch
    validation_loss,validation_accuracy=0,0
    
    for batch_inputs,batch_targets in validation_data:
        
        validation_loss,validation_accuracy=sess.run([mean_loss,accuracy],
                                                     feed_dict={inputs:batch_inputs,targets:batch_targets})
    
    print('Epoch '+str(epoch_counter+1)+
          '. Mean loss: '+'{0:.4f}'.format(current_epoch_loss)+
          '. Validation loss: '+'{0:.4f}'.format(validation_loss)+
          '. Validation accuracy: '+'{0:.3f}'.format(validation_accuracy * 100.)+'%')
    if validation_loss>previous_validation_loss:
        break
    previous_validation_loss=validation_loss
print('end of training')

Epoch 1. Mean loss: 0.8363. Validation loss: 0.7723. Validation accuracy: 52.796%
Epoch 2. Mean loss: 0.7070. Validation loss: 0.6802. Validation accuracy: 58.166%
Epoch 3. Mean loss: 0.6364. Validation loss: 0.6184. Validation accuracy: 61.074%
Epoch 4. Mean loss: 0.5890. Validation loss: 0.5731. Validation accuracy: 63.982%
Epoch 5. Mean loss: 0.5545. Validation loss: 0.5401. Validation accuracy: 66.443%
Epoch 6. Mean loss: 0.5277. Validation loss: 0.5153. Validation accuracy: 71.141%
Epoch 7. Mean loss: 0.5065. Validation loss: 0.4958. Validation accuracy: 73.154%
Epoch 8. Mean loss: 0.4888. Validation loss: 0.4800. Validation accuracy: 74.720%
Epoch 9. Mean loss: 0.4739. Validation loss: 0.4668. Validation accuracy: 75.615%
Epoch 10. Mean loss: 0.4612. Validation loss: 0.4554. Validation accuracy: 75.839%
Epoch 11. Mean loss: 0.4502. Validation loss: 0.4457. Validation accuracy: 76.063%
Epoch 12. Mean loss: 0.4406. Validation loss: 0.4372. Validation accuracy: 76.510%
Epoch 13. Mea

Epoch 103. Mean loss: 0.3175. Validation loss: 0.3299. Validation accuracy: 82.550%
Epoch 104. Mean loss: 0.3173. Validation loss: 0.3298. Validation accuracy: 82.550%
Epoch 105. Mean loss: 0.3170. Validation loss: 0.3296. Validation accuracy: 82.550%
Epoch 106. Mean loss: 0.3168. Validation loss: 0.3295. Validation accuracy: 82.550%
Epoch 107. Mean loss: 0.3166. Validation loss: 0.3294. Validation accuracy: 82.550%
Epoch 108. Mean loss: 0.3163. Validation loss: 0.3293. Validation accuracy: 82.550%
Epoch 109. Mean loss: 0.3161. Validation loss: 0.3292. Validation accuracy: 82.327%
Epoch 110. Mean loss: 0.3159. Validation loss: 0.3291. Validation accuracy: 82.103%
Epoch 111. Mean loss: 0.3156. Validation loss: 0.3290. Validation accuracy: 82.327%
Epoch 112. Mean loss: 0.3154. Validation loss: 0.3289. Validation accuracy: 82.327%
Epoch 113. Mean loss: 0.3152. Validation loss: 0.3288. Validation accuracy: 82.103%
Epoch 114. Mean loss: 0.3150. Validation loss: 0.3287. Validation accuracy: 

In [149]:
test_data=Audiobooks_Data_Reader('test')
test_loss,test_accuracy=0,0
    
for batch_inputs,batch_targets in test_data:
    test_loss,test_accuracy=sess.run([mean_loss,accuracy],
                                     feed_dict={inputs:batch_inputs,targets:batch_targets})
test_accuracy_percent = test_accuracy * 100.
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%'+' . Mean loss: '+'{0:.3f}'.format(test_loss))

Test accuracy: 80.80% . Mean loss: 0.348
